# Get data from the python client

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Swagger UI)[https://control.green-fusion.de/services/energy-management-backend/v1/api/doc#/]


## Inits

In [1]:
#imports
from energy_management_client import BackendPythonClient
import pandas as pd
import json
from datetime import datetime
import pprint  # For pretty-printing

In [2]:
# select your environment
environment="prod" # prod or dev

# Selector
if environment=="dev":
    backend_endpoint_path="https://dev.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="development"
elif environment=="prod":
    backend_endpoint_path="https://control.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="control"  
else:
    backend_endpoint_path=""
    realm_type=""
    print("select either prod or dev for the environment")

#test
#backend_endpoint_path, realm_type
if True:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint=backend_endpoint_path,)
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm=realm_type,
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

## Functions

### buildings

In [3]:
def get_buildings_df(client, customers):
    """
    Loop through all customers, fetch their buildings, and build a DataFrame where each row contains:
      - building_id, customerID (from customer object), address, city, atvise_display_name, base,
      - heatingType, heatingSurface, access_key, nickname, postal_code, gui_name,
      - coordinates, CustomerAlias, gfid,
      - LayoutID (extracted from modular_system)

    Usage:
        customers = client.customers.get_list_of_customers()
        df = get_buildings_df(client, customers)
        print(df.head())
    """
    try:
        data = []
        for c in customers:
            customer_id = c.id
            customer_name = getattr(c, 'name', None)
            client.configure(customer_id=customer_id)
            
            # Get buildings
            c_buildings = client.buildings.get_buildings()
            building_list = getattr(c_buildings, 'buildings', [])
            
            for b in building_list:
                building_id         = getattr(b, 'building_id', None)
                address             = getattr(b, 'address', None)
                city                = getattr(b, 'city', None)
                atvise_display_name = getattr(b, 'atvise_display_name', None)
                base                = getattr(b, 'base', None)
                heatingType         = getattr(b, 'heatingType', None)
                heatingSurface      = getattr(b, 'heatingSurface', None)
                access_key          = getattr(b, 'access_key', None)
                nickname            = getattr(b, 'nickname', None)
                postal_code         = getattr(b, 'postal_code', None)
                gui_name            = getattr(b, 'gui_name', None)
                coordinates         = getattr(b, 'coordinates', None)
                CustomerAlias       = getattr(b, 'CustomerAlias', None)
                gfid                = getattr(b, 'gfid', None)
                
                # Parse modular_system
                modular_system_str = getattr(b, 'modular_system', None)
                LayoutID = None
                if modular_system_str:
                    try:
                        modular_system = json.loads(modular_system_str)
                        LayoutID = modular_system.get('layoutID', None)
                    except json.JSONDecodeError:
                        LayoutID = None

                data.append({
                    "building_id": building_id,
                    "customerID": customer_id,  # <-- corrected here
                    "customer_name": customer_name,
                    "address": address,
                    "city": city,
                    "atvise_display_name": atvise_display_name,
                    "base": base,
                    "heatingType": heatingType,
                    "heatingSurface": heatingSurface,
                    "access_key": access_key,
                    "nickname": nickname,
                    "postal_code": postal_code,
                    "gui_name": gui_name,
                    "coordinates": coordinates,
                    "CustomerAlias": CustomerAlias,
                    "gfid": gfid,
                    "LayoutID": LayoutID
                })

        return pd.DataFrame(data)

    except Exception as e:
        print("API Call Failed:", e)
        return pd.DataFrame()

### devices

In [4]:
def get_devices(client):
    """
    usage:
    list_device=get_devices(client)
    list(list_device)
    """
    devices_list=client.devices.get_devices()
    devices_list=list(devices_list)[0][1]
    device_dicts = [device.__dict__ for device in devices_list]
    df_device = pd.DataFrame(device_dicts)
    return df_device

### counters

In [5]:
def get_counters(client,ID):
    """
    usage:
    counters_list_for_building=get_counters(client,122)
    """
    counters_list=client.buildings.get_counters(ID)
    return counters_list

def get_counters2(client, ID):
    """
    Fetch counter data and extract error information.

    Returns:
        has_error (bool): True if any sensor returns an error.
        error_info (dict): {sensor_id: {'error': str, 'first_occurrence': datetime, 'last_occurrence': datetime}}
    example:
        get_counters2(client,122)
    """
    counters_list = client.buildings.get_counters(ID)

    error_info = {}

    for counter in counters_list:
        if counter.error and counter.error.strip().lower() != 'none':
            sensor_id = counter.sensor_id
            ref_date = counter.reference_date

            if sensor_id not in error_info:
                error_info[sensor_id] = {
                    'error': counter.error,
                    'first_occurrence': ref_date,
                    'last_occurrence': ref_date,
                    'short_name': counter.short_name,
                    'long_name': counter.long_name,
                    'hydraulic_location_index': counter.hydraulic_location_index
                }
            else:
                if ref_date < error_info[sensor_id]['first_occurrence']:
                    error_info[sensor_id]['first_occurrence'] = ref_date
                if ref_date > error_info[sensor_id]['last_occurrence']:
                    error_info[sensor_id]['last_occurrence'] = ref_date

    has_error = len(error_info) > 0
    return has_error, error_info

In [6]:
#get_counters2(client,122)

## get_buildings_df

In [7]:
# Get customers list (list of dicts)
customers = client.customers.get_list_of_customers()
#build df
df_buildings = get_buildings_df(client, customers)
#test
df_buildings.head()

,building_id,customerID,customer_name,address,city,atvise_display_name,base,heatingType,heatingSurface,access_key,nickname,postal_code,gui_name,coordinates,CustomerAlias,gfid,LayoutID
0,756,66,HwS,Wolzogenstr.28,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,14163,Hilfswerk-Siedlung GmbH,"[52.432631, 13.234862]",None,GFP-12505,gas:1--gas:2--heating-circuit:1--heating-circu...
1,757,66,HwS,Hagelberger Str. 26,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,10965,Hilfswerk-Siedlung GmbH,"[52.491932, 13.379842]",None,GFP-13438,gas:1--gas:2--heating-circuit:1
2,758,66,HwS,Hochstr. 8,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,13357,Hilfswerk-Siedlung GmbH,"[52.54765, 13.381522]",None,GFP-12475,gas:1--heating-circuit:1
3,759,66,HwS,Planufer 82a,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,10967,Hilfswerk-Siedlung GmbH,"[52.494745, 13.413277]",None,GFP-12530,gas:1--heating-circuit:1
4,761,66,HwS,Gabainstr. 13,Berlin,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,12247,Hilfswerk-Siedlung GmbH,"[52.436169, 13.360646]",None,GFP-12605,gas:1--gas:2--global-separation-circuit:1--hea...


In [8]:
df_buildings[['building_id', 'customerID','gfid','LayoutID']].head()

,building_id,customerID,gfid,LayoutID
0,756,66,GFP-12505,gas:1--gas:2--heating-circuit:1--heating-circu...
1,757,66,GFP-13438,gas:1--gas:2--heating-circuit:1
2,758,66,GFP-12475,gas:1--heating-circuit:1
3,759,66,GFP-12530,gas:1--heating-circuit:1
4,761,66,GFP-12605,gas:1--gas:2--global-separation-circuit:1--hea...


In [9]:
df_buildings.count()

building_id            1935
customerID             1935
customer_name          1935
address                1935
city                   1920
atvise_display_name      13
base                     13
heatingType               0
heatingSurface            0
access_key             1935
nickname                216
postal_code            1920
gui_name               1935
coordinates            1935
CustomerAlias             0
gfid                   1475
LayoutID               1137
dtype: int64

## get devices

In [10]:
df_devices=get_devices(client)
df_devices = df_devices.rename(columns={"type": "device_type"})

In [11]:
df_devices.head()

,device_type,building_id,device_id
0,RUT956,393,23515803
1,RUT956,444,bbulentze107
2,RUT956,627,2351004
3,RUT956,563,2351007
4,RUT956,533,2346008


In [12]:
df_devices.count()

device_type    1068
building_id    1068
device_id      1068
dtype: int64

## Merge Dfs

In [13]:
# Merge the two DataFrames on building_id
df_merged = df_buildings.merge(
    df_devices[['building_id', 'device_type', 'device_id']],
    on='building_id',
    how='left'  # ensures all rows from df_buildings are kept
)

In [14]:
df_merged.columns

Index(['building_id', 'customerID', 'customer_name', 'address', 'city',
       'atvise_display_name', 'base', 'heatingType', 'heatingSurface',
       'access_key', 'nickname', 'postal_code', 'gui_name', 'coordinates',
       'CustomerAlias', 'gfid', 'LayoutID', 'device_type', 'device_id'],
      dtype='object')

# explore data

In [15]:
def get_building_details(client, customer_id, building_id):
    """
    Fetch and display detailed information for a specific building.

    Parameters:
        client: An API client instance configured for use.
        customer_id (int): The ID of the customer to which the building belongs.
        building_id (int): The ID of the building to retrieve.

    Returns:
        dict: A dictionary of building attributes, displayed nicely.
    
    Example:
        get_building_details(client, customer_id=42, building_id=101)
    """
    # Configure the client for the selected customer
    client.configure(customer_id=customer_id)
    
    # Get all buildings for this customer
    buildings_response = client.buildings.get_buildings()
    buildings = getattr(buildings_response, 'buildings', [])

    # Find the building with the matching ID
    building = next((b for b in buildings if getattr(b, 'building_id', None) == building_id), None)

    if not building:
        print(f"No building found with ID {building_id} for customer {customer_id}")
        return None

    # Extract fields
    details = {
        "building_id": getattr(building, 'building_id', None),
        "customerID": getattr(building, 'customerID', None),
        "address": getattr(building, 'address', None),
        "city": getattr(building, 'city', None),
        "atvise_display_name": getattr(building, 'atvise_display_name', None),
        "base": getattr(building, 'base', None),
        "heatingType": getattr(building, 'heatingType', None),
        "heatingSurface": getattr(building, 'heatingSurface', None),
        "access_key": getattr(building, 'access_key', None),
        "nickname": getattr(building, 'nickname', None),
        "postal_code": getattr(building, 'postal_code', None),
        "gui_name": getattr(building, 'gui_name', None),
        "coordinates": getattr(building, 'coordinates', None),
        "CustomerAlias": getattr(building, 'CustomerAlias', None),
        "gfid": getattr(building, 'gfid', None),
    }

    # Parse modular_system if present
    modular_system_str = getattr(building, 'modular_system', None)
    if modular_system_str:
        try:
            modular_system = json.loads(modular_system_str)
            details["modular_system"] = modular_system
        except json.JSONDecodeError:
            details["modular_system"] = "Invalid JSON"

    # Display nicely
    pprint.pprint(details, sort_dicts=False)

    return details


In [16]:
df_merged[df_merged['building_id'] == 1801]

,building_id,customerID,customer_name,address,city,atvise_display_name,base,heatingType,heatingSurface,access_key,nickname,postal_code,gui_name,coordinates,CustomerAlias,gfid,LayoutID,device_type,device_id
624,1801,118,Wohnungsgenossenschaft von 1904,Buchsbaumweg 11,Hamburg,None,None,None,None,&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71...,None,22299,Wohnungsgenossenschaft von 1904 e. G.,"[53.601105, 9.997589]",None,GFP-22303,district-heating:1--heat-exchanger:1-heating-c...,RUT956,2504033


In [17]:
print(get_building_details(client, customer_id=118, building_id=1801))

{'building_id': 1801,
 'customerID': None,
 'address': 'Buchsbaumweg 11',
 'city': 'Hamburg',
 'atvise_display_name': None,
 'base': None,
 'heatingType': None,
 'heatingSurface': None,
 'access_key': '&atvise_key=91e70eae-b284-4fd4-b124-72b76d4f71ed&atvise_token=c1be3026-22ba-4851-b9fa-c5214b3a79f1',
 'nickname': None,
 'postal_code': '22299',
 'gui_name': 'Wohnungsgenossenschaft von 1904 e. G.',
 'coordinates': ['53.601105', '9.997589'],
 'CustomerAlias': None,
 'gfid': 'GFP-22303',
 'modular_system': {'name': 'modular-system-entry',
                    'id': 'modular-system-mock-1',
                    'buildingID': 1801,
                    'version': 'v1',
                    'published': True,
                    'layoutID': 'district-heating:1--heat-exchanger:1-heating-circuit:1--heating-circuit:2',
                    'subSystems': [{'id': 'heating-circuit:1',
                                    'name': 'heating-circuit',
                                    'hydraulicLocationIn

## Exports df to csv

In [18]:
if True:
    print(list(df_merged.columns))

['building_id', 'customerID', 'customer_name', 'address', 'city', 'atvise_display_name', 'base', 'heatingType', 'heatingSurface', 'access_key', 'nickname', 'postal_code', 'gui_name', 'coordinates', 'CustomerAlias', 'gfid', 'LayoutID', 'device_type', 'device_id']


In [19]:
# get_buildings_df
if True:
    today_str = datetime.today().strftime('%Y-%m-%d')
    filename = f"data_from_db_{today_str}.csv"
    df_merged[['building_id',
               'customerID',
               'customer_name',
               'address',
               'postal_code',
               'city',
               #'heatingType',
               #'heatingSurface',
               #'coordinates',
               'gfid',
               'LayoutID',
               'device_type',
               'device_id']].to_csv(filename, index=False)